In [ ]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers pypdf chroma gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## Initializing the Hugging Face Pipeline

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, you need an access token
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# hf_auth = user_secrets.get_secret("hf_token")
hf_auth = ""

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    token=hf_auth
)

stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    # temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
res = generate_text("What are the key features of Oracle Cloud Object Storage?")
print(res[0]["generated_text"])

What are the key features of Oracle Cloud Object Storage?
 Unterscheidung between cloud computing and object storage. 15 Apr 2022 Oracle Cloud Infrastructure (OCI) is a comprehensive cloud computing platform that provides a wide range of services, including computing, storage, networking, and security. Within OCI, Oracle Cloud Object Storage is a scalable, durable, and secure cloud-based storage service that allows users to store and retrieve large amounts of unstructured data, such as videos, images, and documents. Here are some key features of Oracle Cloud Object Storage: Scalability: Oracle Cloud Object Storage is designed to scale with your needs, providing you with the capacity you require to store and manage massive amounts of unstructured data. Durability: The service ensures high availability and redundancy to protect against data loss due to hardware failures or other disruptions. Security: Oracle Cloud Object Storage provides advanced security features, including encryption a

In [ ]:
res

[{'generated_text': 'What are the key features of Oracle Cloud Object Storage?\n Unterscheidung between cloud computing and object storage. 15 Apr 2022 Oracle Cloud Infrastructure (OCI) is a comprehensive cloud computing platform that provides a wide range of services, including computing, storage, networking, and security. Within OCI, Oracle Cloud Object Storage is a scalable, durable, and secure cloud-based storage service that allows users to store and retrieve large amounts of unstructured data, such as videos, images, and documents. Here are some key features of Oracle Cloud Object Storage: Scalability: Oracle Cloud Object Storage is designed to scale with your needs, providing you with the capacity you require to store and manage massive amounts of unstructured data. Durability: The service ensures high availability and redundancy to protect against data loss due to hardware failures or other disruptions. Security: Oracle Cloud Object Storage provides advanced security features, 

## Implementing HF Pipeline in LangChain

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
llm(prompt="What are the key features of Oracle Cloud Object Storage?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n Unterscheidung between cloud computing and object storage in terms of their features, benefits and drawbacks.  A comparison of Amazon S3, Microsoft Azure Blob Storage, Google Cloud Storage, and Oracle Cloud Object Storage based on their features, pricing, performance, and security.'

## Ingesting Data using Document Loader

In [ ]:
## Needed for Google Colab
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Download the OCI user guide
!wget https://www.oracle.com/oce/dc/assets/CONT1B79FBD79BF543B18459A353101B0F91/native/oci-user-guide.pdf

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/oci-user-guide.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

--2024-01-18 06:36:25--  https://www.oracle.com/oce/dc/assets/CONT1B79FBD79BF543B18459A353101B0F91/native/oci-user-guide.pdf
Resolving www.oracle.com (www.oracle.com)... 23.35.140.25, 2600:1407:a800:586::a15, 2600:1407:a800:5ac::a15
Connecting to www.oracle.com (www.oracle.com)|23.35.140.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57728504 (55M) [application/pdf]
Saving to: ‘oci-user-guide.pdf’

oci-user-guide.pdf  100%[===================>]  55.05M  16.4MB/s    in 3.4s    

2024-01-18 06:36:29 (16.4 MB/s) - ‘oci-user-guide.pdf’ saved [57728504/57728504]



In [ ]:
len(docs)

5787

In [ ]:
docs[0]

Document(page_content='User Guide\nOctober 12, 2021', metadata={'source': '/content/oci-user-guide.pdf', 'page': 0})

## Splitting text

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

12327

## Creating Embedding and Storing in VectorDB

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectordb = FAISS.from_documents(splits, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
question = "what is oci data science?"
docs = vectordb.similarity_search(question,k=1)
print(docs[0].page_content)

Data Science  is a platform for data scientists to build, train, and manage machine learning models on Oracle Cloud
Infrastructure , using Python and open source machine learning libraries. Teams of data scientists can organize their
work and access data and computing resources in this collaborative environment.
Data Transfer  lets you migrate large volumes of data to Oracle Cloud Infrastructure .
Database  lets you easily build, scale, and secure Oracle databases with license-included pricing in the Oracle
cloud. You can also leverage Oracle Cloud Infrastructure  to manage Oracle databases in your data center alongside
your cloud databases. OCI's Autonomous Database  delivers automated patching, upgrades, and tuning on shared
or dedicated cloud Exadata infrastructure. Autonomous Database  is also available in your data center with OCI's
Exadata Cloud@Customer  service. OCI's Exadata Cloud Service  allows you to leverage the power of scalable
Exadata hardware in the Oracle cloud. OCI's

## Init Chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(),
    return_source_documents=True)

In [ ]:
chat_history = []
def chat_with_oci(query):
    result = chain({"question": query, "chat_history": chat_history})['answer'].rstrip().split('\n')

    #print(result['answer'])
    return result

In [ ]:
query = "What are the key features of Oracle Cloud Object Storage?"
chat_with_oci(query)

[' Based on the provided text, the key features of Oracle Cloud Object Storage include flexible storage for unstructured data, strong encryption, deep integration with Identity and Access Management, and support for multiple management interfaces.']

In [ ]:
query = "Explain the steps to create a virtual machine instance in Oracle Cloud."
chat_with_oci(query)

[' To create a virtual machine instance in Oracle Cloud, follow these steps:',
 '',
 'Step 1: Open the navigation menu and click on "Compute". Under "Compute", click on "Instances".',
 'Step 2: Click on "Create Instance" to start creating a new instance. Enter a name for the instance and select the compartment where you want to create it.',
 'Step 3: In the "Placement" section, select the availability domain where you want to create the instance.',
 'Step 4: Select the type of instance you want to create, such as a Linux instance. If you want to use your own SSH key to connect to the instance via SSH, provide the public key from the SSH key pair.',
 'Step 5: Review the summary of the instance creation and click on "Create Instance" to finalize the creation process.',
 '',
 'Alternatively, if you have an existing VCN, you can launch the instance directly within the VCN without creating a new one. Just select the VCN as the placement option during the instance creation process.']

In [ ]:
query = "Tell me about the security measures in Oracle Cloud."
chat_with_oci(query)

[" Oracle Cloud Infrastructure takes security very seriously, and they have implemented several measures to ensure the safety of their customers' data. These include:",
 '',
 "* Customer isolation: Oracle Cloud Infrastructure uses a multi-tenant architecture that ensures each customer's resources are fully isolated from those of other customers, as well as from Oracle's staff.",
 '* Always-on encryption: Oracle Cloud Infrastructure encrypts all data at rest and in transit, ensuring that customer data is protected even when it is not being actively accessed.',
 '* Easy-to-use IAM policies: Oracle Cloud Infrastructure provides easy-to-use identity and access management (IAM) policies that allow customers to constrain access to their services and segregate operational responsibilities, reducing the risk associated with malicious or accidental user actions.',
 '* Security zones: Oracle Cloud Infrastructure divides its infrastructure into security zones, which are isolated from one another.

In [ ]:
query = "What services does Oracle Cloud offer for database management?"
chat_with_oci(query)

[' According to the passage, Oracle Cloud offers the following services for database management:',
 '',
 '* Database Management: Provides comprehensive database performance diagnostics and management capabilities to monitor and manage Oracle databases.',
 '* Database Migration: Is a fully managed service that helps database administrators move databases in real-time, at scale, from one or more source databases to Oracle Cloud databases.',
 '',
 "Unhelpful Answer: I don't know."]

In [ ]:
query = "What is Oracle Cloud Identity and Access Management (IAM) and how does it work?"
chat_with_oci(query)

[' Oracle Cloud Identity and Access Management (IAM) is a service provided by Oracle Cloud Infrastructure that enables organizations to securely manage access to their cloud resources. IAM works by creating and managing users, groups, and resources within a logical structure called a compartment. Resources are grouped into compartments, which provide isolation and security. IAM also includes security policies that define the types of access different groups have to resources within a specified aggregation level, such as a tenancy, compartment, or service. Additionally, IAM includes dynamic groups that allow for the creation of groups of compute instances as principal actors, enabling instances to make API calls to Oracle Cloud Infrastructure. Finally, IAM includes tags that allow for the organization of resources across multiple compartments for reporting purposes or for taking bulk actions.']

In [ ]:
query = "How does Oracle Cloud support disaster recovery?"
chat_with_oci(query)

[' Oracle Cloud Infrastructure supports disaster recovery through its built-in high availability features and Maximum Availability Architecture best practices. Autonomous Database Cloud instances are highly available by default, with a multi-node configuration to protect against localized hardware failures. Major database upgrades require downtime, but automatic backups are stored in Oracle Cloud Infrastructure Object Storage and replicated to another availability domain for disaster recovery. Additionally, Oracle Cloud Infrastructure provides notifications of any VM infrastructure failure events and comprehensive database performance diagnostics and management capabilities to monitor and manage Oracle databases.']

In [ ]:
query = "Tell me about the billing and cost management features in Oracle Cloud."
chat_with_oci(query)

[' Oracle Cloud offers several billing and cost management features, including budgets, cost analysis, and cost and usage reports. Budgets allow you to set thresholds for your spending and receive alerts when you might exceed your budget. Cost analysis provides visualization tools to track and optimize your spending, while cost and usage reports provide detailed breakdowns of your invoice line items at resource-level granularity. Additionally, Oracle Cloud offers unified billing across multiple tenancies by sharing subscriptions between tenancies.']